# 事前知識

### 1次元の畳み込み

In [19]:
import numpy as np

def conv1d(x,w,p=0,s=0):
    w_rot = np.array(w[::-1]) #反転
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_padded = np.concatenate([zero_pad, x_padded, zero_pad])
    res = []
    for i in range(0, int(len(x)/s), s):
        res.append(np.sum(x_padded[i:i+w_rot.shape[0]] * w_rot))
    return np.array(res)

## Testing:
x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]
print('Conv1d Implementation: ', 
      conv1d(x, w, p=2, s=1)) # p:ゼロ埋め、s:フィルタのシフト
print('Numpy Results:         ', 
      np.convolve(x, w, mode='same'))

Conv1d Implementation:  [ 5. 14. 16. 26. 24. 34. 19. 22.]
Numpy Results:          [ 5 14 16 26 24 34 19 22]


### 2次元の畳み込み

In [20]:
import numpy as np
import scipy.signal

def conv2d(X, W, p=(0, 0), s=(1, 1)):
    W_rot = np.array(W)[::-1,::-1]
    X_orig = np.array(X)
    n1 = X_orig.shape[0] + 2 * p[0]
    n2 = X_orig.shape[1] + 2 * p[1]
    X_padded = np.zeros(shape=(n1 ,n2))
    X_padded[p[0]:p[0]+X_orig.shape[0],
             p[1]:p[1]+X_orig.shape[1]] = X_orig
    res = []
    for i in range(0, int((X_padded.shape[0] - W_rot.shape[0])/s[0])+1, s[0]):
        res.append([])
        for j in range(0, int((X_padded.shape[1] - W_rot.shape[1])/s[1])+1, s[1]):
            X_sub = X_padded[i:i+W_rot.shape[0], j:j+W_rot.shape[1]]
            res[-1].append(np.sum(X_sub * W_rot))
    return(np.array(res))

In [21]:
X = [[1, 3, 2, 4], [5, 6, 1, 3], [1 , 2,0, 2], [3, 4, 3, 2]]
W = [[1, 0, 3], [1, 2, 1], [0, 1, 1]]
print('Conv2d Implementation: \n', 
      conv2d(X, W, p=(1,1), s=(1,1)))

print('Scipy Results:         \n', 
      scipy.signal.convolve2d(X, W, mode='same'))

Conv2d Implementation: 
 [[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]
Scipy Results:         
 [[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]


### RGB画像の読み込み

In [22]:
import imageio
try:
    img = imageio.imread('./example-image.png', pilmode='RGB')
except AttributeError:
    s = ("imageio.imread requires Python's image library PIL"
         " You can satisfy this requirement by installing the"
         " userfriendly fork PILLOW via `pip install pillow`.")
    raise AttributeError(s)

print('Image shape:', img.shape)
print('Number of channels:', img.shape[2])
print('Image data type:', img.dtype)
print(img[100:102, 100:102, :])

Image shape: (252, 221, 3)
Number of channels: 3
Image data type: uint8
[[[179 134 110]
  [182 136 112]]

 [[180 135 111]
  [182 137 113]]]


In [24]:
print(img.shape[0])
print(img.shape[1])
print(img.shape[2])

252
221
3


# CNNの実装

### minstファイルの読み込み

In [95]:
import sys
import gzip
import shutil
import os

In [96]:
#zipファイルのリストの取得
if (sys.version_info > (3, 0)):
    writemode = 'wb'
else:
    writemode = 'w'

zipped_mnist_tmp = [f for f in os.listdir('./mnist2/') if f.endswith('ubyte.gz')]
zipped_mnist_tmp

['t10k-images-idx3-ubyte.gz',
 't10k-labels-idx1-ubyte.gz',
 'train-images-idx3-ubyte.gz',
 'train-labels-idx1-ubyte.gz']

In [97]:
# Listにファイルパスを追記
zipped_mnist = []
for file_name in zipped_mnist_tmp:
    file_path = './mnist2/' + file_name
    zipped_mnist.append(file_path)
zipped_mnist

['./mnist2/t10k-images-idx3-ubyte.gz',
 './mnist2/t10k-labels-idx1-ubyte.gz',
 './mnist2/train-images-idx3-ubyte.gz',
 './mnist2/train-labels-idx1-ubyte.gz']

In [98]:
#gzファイルを解凍
for z in zipped_mnist:
    with gzip.GzipFile(z, mode='rb') as decompressed, open(z[:-3], writemode) as outfile:
        outfile.write(decompressed.read())

In [99]:
# load関数を定義
import struct
import numpy as np

def load_mnist(path, kind='train'):
    labels_path = os.path.join(path,'%s-labels-idx1-ubyte'%kind)
    images_path = os.path.join(path,'%s-images-idx3-ubyte'%kind)
    
    with open(labels_path, 'rb') as lbpath:
        # big endian::unsigned int::unsigned int
        # 頭の8バイトはヘッダ
        magic, n = struct.unpack('>II',lbpath.read(8))
        # 8ビットの符号なし整数
        labels = np.fromfile(lbpath, dtype=np.uint8)
        
    with open(images_path,'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII", imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
    
    return images, labels

In [119]:
# データの読み込み
# 784 = 28×28
X_data, y_data = load_mnist('./mnist2/', kind='train')
print('Rows: %d,  Columns: %d' % (X_data.shape[0], X_data.shape[1]))
X_test, y_test = load_mnist('./mnist2/', kind='t10k')
print('Rows: %d,  Columns: %d' % (X_test.shape[0], X_test.shape[1]))

X_train, y_train = X_data[:50000,:], y_data[:50000]
X_valid, y_valid = X_data[50000:,:], y_data[50000:]

print('Training:   ', X_train.shape, y_train.shape)
print('Validation: ', X_valid.shape, y_valid.shape)
print('Test Set:   ', X_test.shape, y_test.shape)

Rows: 60000,  Columns: 784
Rows: 10000,  Columns: 784
Training:    (50000, 784) (50000,)
Validation:  (10000, 784) (10000,)
Test Set:    (10000, 784) (10000,)


### ミニバッチ

In [101]:
# データをミニバッチごとに処理する関数の定義
def batch_generator(X, y, batch_size=64, shuffle=False, random_seed=None):
    # 1ずつの等差数列を作成
    idx = np.arange(y.shape[0])
    
    # INDEXのシャッフル
    if shuffle:
        rng = np.random.RandomState(random_seed)
        rng.shuffle(idx)
        X = X[idx]
        y = y[idx]
    
    for i in range(0, X.shape[0], batch_size):
        yield ( X[i:i+batch_size, :], y[i:i+batch_size])

### 正規化

In [120]:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_train_centered = (X_train - mean_vals)/std_val
X_valid_centered = (X_valid - mean_vals)/std_val
X_test_centered = (X_test - mean_vals)/std_val


### CNNの実装(畳み込み層)

In [103]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

# 畳み込み層のラッパー関数
def conv_layer(
    input_tensor              #インプットテンソル
    ,name                     # 名前
    ,kernel_size              # フィルタサイズ
    ,n_output_channels        # 出力数
    ,padding_mode = 'SAME'    # full,same,valid
    ,strides=(1, 1, 1, 1)):   # スライド数
    
    # tf.variable_scope :: 変数（識別子）を管理するための専用のスコープ定義
    # tf.get_variable   :: (<変数名>)
    # tf.Variable      ::  (<初期値><変数名>) 
    
    with tf.variable_scope(name):
        # 入力チャネルの個数を取得
        # input_tensorの形状 :: バッチ×幅×高さ×チャネル数
        input_shape = input_tensor.get_shape().as_list()
        n_input_channels = input_shape[-1] 
        
        weights_shape = (list(kernel_size) + [n_input_channels, n_output_channels]) # [3,3,3,3]
        weights = tf.get_variable(
            name='_weights'
            ,shape=weights_shape)
        print(weights)
        
        biases = tf.get_variable(
            name='_biases'
            ,initializer=tf.zeros(shape=[n_output_channels])) # 初期値は0
        print(biases)
        
        # conv :: 畳み込みもしくはプーリング層の中間出力結果
        # tf.nn.conv2d :: 畳み込み層で利用
        conv = tf.nn.conv2d(
            input=input_tensor
            ,filter=weights
            ,strides=strides 
            ,padding=padding_mode)
        print(conv)
        
        # tf.nn.bias_add :: 重みを積和したものにバイアスを加算する際に利用
        conv = tf.nn.bias_add(conv,biases,name='net_pre-activation')
        print(conv)
        
        # tf.nn.relu :: 畳み込み層のアクティベーション関数としてReLuを利用
        conv = tf.nn.relu(conv, name='activation')
        print(conv)
        
        return conv

In [104]:
## testing
g = tf.Graph()
with g.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
    conv_layer(x, name='convtest', kernel_size=(3, 3), n_output_channels=32)

<tf.Variable 'convtest/_weights:0' shape=(3, 3, 1, 32) dtype=float32_ref>
<tf.Variable 'convtest/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("convtest/Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("convtest/net_pre-activation:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("convtest/activation:0", shape=(?, 28, 28, 32), dtype=float32)


### CNNの実装(全結合層)

In [105]:
# get_shapeの検証
xx = tf.placeholder(tf.float32, shape=[None, 50, 28, 1])
xx.get_shape().as_list()[1:]

[50, 28, 1]

In [106]:
def fc_layer(
    input_tensor            # インプットテンソル
    ,name                   # 名前
    ,n_output_units         # 出力ユニット数
    ,activation_fn=None):
    
    with tf.variable_scope(name):
        input_shape = input_tensor.get_shape().as_list()[1:] # テンソルのインデックス1以降をリスト化
        n_input_units = np.prod(input_shape)                 # np.prod() :: 配列要素の積
        
        if len(input_shape) > 1:
            input_tensor = tf.reshape(input_tensor,shape=(-1, n_input_units))
        
        # 重み
        weights_shape = [n_input_units, n_output_units]
        weights = tf.get_variable(name='_weights',shape=weights_shape)
        print(weights)
        
        # バイアス
        biases = tf.get_variable(name='_biases',initializer=tf.zeros(shape=[n_output_units]))
        print(biases)
        
        # 行列積
        layer = tf.matmul(input_tensor, weights)
        print(layer)
        
        # バイアスの加算
        layer = tf.nn.bias_add(layer, biases,name='net_pre-activation')
        print(layer)
        
        # 活性化関数
        if activation_fn is None:
            return layer
        
        layer = activation_fn(layer, name='activation')
        print(layer)
        return layer

In [107]:
g = tf.Graph()
with g.as_default():
    x = tf.placeholder(tf.float32, 
                       shape=[None, 28, 28, 1])
    fc_layer(x, name='fctest', n_output_units=32, 
             activation_fn=tf.nn.relu)

<tf.Variable 'fctest/_weights:0' shape=(784, 32) dtype=float32_ref>
<tf.Variable 'fctest/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("fctest/MatMul:0", shape=(?, 32), dtype=float32)
Tensor("fctest/net_pre-activation:0", shape=(?, 32), dtype=float32)
Tensor("fctest/activation:0", shape=(?, 32), dtype=float32)


### CNNの構築

In [114]:
def build_cnn(learning_rate=1e-4):
    # Xとyのプレースホルダ―を作成
    tf_x = tf.placeholder(tf.float32, shape=[None, 784], name='tf_x')
    tf_y = tf.placeholder(tf.int32, shape=[None], name='tf_y')
    
    # xを4次元テンソルに変換 :: [バッチサイズ, 幅, 高さ, 1]
    tf_x_image = tf.reshape(tf_x, shape=[-1, 28, 28, 1], name='tf_x_reshaped')
    
    # one-hotエンコーディング
    tf_y_onehot = tf.one_hot(
        indices=tf_y
        ,depth=10
        ,dtype=tf.float32
        ,name='tf_y_onehot'
    )
    
    ## 1st layer: 畳み込み層
    print('\nBuilding 1st layer: ')
    h1 = conv_layer(tf_x_image, name='conv_1',
                    kernel_size=(5, 5), 
                    padding_mode='VALID',
                    n_output_channels=32)
    
    ## 最大値プーリング
    h1_pool = tf.nn.max_pool(h1, 
                             ksize=[1, 2, 2, 1],
                             strides=[1, 2, 2, 1], 
                             padding='SAME')
    ## 2n layer: 畳み込み層
    print('\nBuilding 2nd layer: ')
    h2 = conv_layer(h1_pool, name='conv_2', 
                    kernel_size=(5,5), 
                    padding_mode='VALID',
                    n_output_channels=64)
    
    ## 最大値プーリング
    h2_pool = tf.nn.max_pool(h2, 
                             ksize=[1, 2, 2, 1],
                             strides=[1, 2, 2, 1], 
                             padding='SAME')

    ## 3rd layer: 全結合層
    print('\nBuilding 3rd layer:')
    h3 = fc_layer(h2_pool, name='fc_3',
                  n_output_units=1024, 
                  activation_fn=tf.nn.relu)

    ## ドロップアウト :: 過学習防止のための仕組み :: 必要がなさそうなノードは削除する
    # keep_prob :: 削除率
    keep_prob = tf.placeholder(tf.float32, name='fc_keep_prob')
    h3_drop = tf.nn.dropout(h3, keep_prob=keep_prob, 
                            name='dropout_layer')

    ## 4th layer: 全結合層(線形活性化)
    print('\nBuilding 4th layer:')
    h4 = fc_layer(h3_drop, name='fc_4',
                  n_output_units=10, 
                  activation_fn=None)

    ## 予測
    predictions = {
        'probabilities' : tf.nn.softmax(h4, name='probabilities'),
        'labels' : tf.cast(tf.argmax(h4, axis=1), tf.int32,
                           name='labels')
    }
    
    ## TensorBoardで計算グラフを可視化

    ## 損失関数と最適化
    # tf.reduce_mean :: 与えたリストに入っている数値の平均値を求める関数
    # tf.nn.softmax_cross_entropy_with_logits :: 基本的に推計結果のsoftmax値を計算して、クロスエントロピーを算出
    # ソフトマックス関数 :: 出力を正規化（０から１に収める）をする
    # クロスエントロピー :: 集合Aと集合Bの乱雑さの度合い
    cross_entropy_loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits=h4, labels=tf_y_onehot),
        name='cross_entropy_loss')

    ## オプティマイザ: 最適化アルゴリズム :: 学習をより高速により効率よく収束させる方法
    optimizer = tf.train.AdamOptimizer(learning_rate)
    optimizer = optimizer.minimize(cross_entropy_loss,
                                   name='train_op')

    ## 予測正解率を特定
    correct_predictions = tf.equal(
        predictions['labels']
        , tf_y
        , name='correct_preds')

    accuracy = tf.reduce_mean(
        tf.cast(correct_predictions, tf.float32)
        ,name='accuracy')

In [109]:
def save(saver, sess, epoch, path='./model/'):
    if not os.path.isdir(path):
        os.makedirs(path)
    print('Saving model in %s' % path)
    saver.save(sess, os.path.join(path,'cnn-model.ckpt'),global_step=epoch)

def load(saver, sess, path, epoch):
    print('Loading model from %s' % path)
    saver.restore(sess, os.path.join(path, 'cnn-model.ckpt-%d' % epoch))

In [111]:
def train(sess, training_set, validation_set=None,
          initialize=True, epochs=20, shuffle=True,
          dropout=0.5, random_seed=None):

    X_data = np.array(training_set[0])
    y_data = np.array(training_set[1])
    training_loss = []

    ## initialize variables
    if initialize:
        sess.run(tf.global_variables_initializer())

    np.random.seed(random_seed) # for shuflling in batch_generator
    for epoch in range(1, epochs+1):
        batch_gen = batch_generator(
                        X_data, y_data, 
                        shuffle=shuffle)
        avg_loss = 0.0
        for i,(batch_x,batch_y) in enumerate(batch_gen):
            feed = {'tf_x:0': batch_x, 
                    'tf_y:0': batch_y, 
                    'fc_keep_prob:0': dropout}
            loss, _ = sess.run(
                    ['cross_entropy_loss:0', 'train_op'],
                    feed_dict=feed)
            avg_loss += loss

        training_loss.append(avg_loss / (i+1))
        print('Epoch %02d Training Avg. Loss: %7.3f' % (
            epoch, avg_loss), end=' ')
        if validation_set is not None:
            feed = {'tf_x:0': validation_set[0],
                    'tf_y:0': validation_set[1],
                    'fc_keep_prob:0':1.0}
            valid_acc = sess.run('accuracy:0', feed_dict=feed)
            print(' Validation Acc: %7.3f' % valid_acc)
        else:
            print()

            
def predict(sess, X_test, return_proba=False):
    feed = {'tf_x:0': X_test, 
            'fc_keep_prob:0': 1.0}
    if return_proba:
        return sess.run('probabilities:0', feed_dict=feed)
    else:
        return sess.run('labels:0', feed_dict=feed)

In [112]:
# seed値を固定
random_seed = 123
np.random.seed(random_seed)

In [115]:
# tfで計算グラフを作成
g = tf.Graph()
with g.as_default():
    tf.set_random_seed(random_seed)
    ## build the graph
    build_cnn()

    ## saver:
    saver = tf.train.Saver()


Building 1st layer: 
<tf.Variable 'conv_1/_weights:0' shape=(5, 5, 1, 32) dtype=float32_ref>
<tf.Variable 'conv_1/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("conv_1/Conv2D:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/net_pre-activation:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/activation:0", shape=(?, 24, 24, 32), dtype=float32)

Building 2nd layer: 
<tf.Variable 'conv_2/_weights:0' shape=(5, 5, 32, 64) dtype=float32_ref>
<tf.Variable 'conv_2/_biases:0' shape=(64,) dtype=float32_ref>
Tensor("conv_2/Conv2D:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/net_pre-activation:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/activation:0", shape=(?, 8, 8, 64), dtype=float32)

Building 3rd layer:
<tf.Variable 'fc_3/_weights:0' shape=(1024, 1024) dtype=float32_ref>
<tf.Variable 'fc_3/_biases:0' shape=(1024,) dtype=float32_ref>
Tensor("fc_3/MatMul:0", shape=(?, 1024), dtype=float32)
Tensor("fc_3/net_pre-activation:0", shape=(?, 1024), dtype=float32

In [121]:
# CNNモデルのトレーニングを実施
with tf.Session(graph=g) as sess:
    train(sess, 
          training_set=(X_train_centered, y_train), 
          validation_set=(X_valid_centered, y_valid), 
          initialize=True,
          random_seed=123)
    save(saver, sess, epoch=20)

Epoch 01 Training Avg. Loss: 272.265  Validation Acc:   0.973
Epoch 02 Training Avg. Loss:  76.135  Validation Acc:   0.983
Epoch 03 Training Avg. Loss:  51.042  Validation Acc:   0.987
Epoch 04 Training Avg. Loss:  40.399  Validation Acc:   0.987
Epoch 05 Training Avg. Loss:  33.019  Validation Acc:   0.988
Epoch 06 Training Avg. Loss:  26.448  Validation Acc:   0.989
Epoch 07 Training Avg. Loss:  23.095  Validation Acc:   0.989
Epoch 08 Training Avg. Loss:  19.834  Validation Acc:   0.990
Epoch 09 Training Avg. Loss:  17.755  Validation Acc:   0.990
Epoch 10 Training Avg. Loss:  14.942  Validation Acc:   0.991
Epoch 11 Training Avg. Loss:  13.460  Validation Acc:   0.990
Epoch 12 Training Avg. Loss:  11.531  Validation Acc:   0.991
Epoch 13 Training Avg. Loss:  10.258  Validation Acc:   0.992
Epoch 14 Training Avg. Loss:   8.869  Validation Acc:   0.992
Epoch 15 Training Avg. Loss:   7.698  Validation Acc:   0.990
Epoch 16 Training Avg. Loss:   7.373  Validation Acc:   0.990
Epoch 17

In [123]:
# 既存のセッションを削除して新しいセッションを作成
del g

g2 = tf.Graph()

with g2.as_default():
    tf.set_random_seed(random_seed)
    build_cnn()
    saver = tf.train.Saver()

# モデルを復元
with tf.Session(graph=g2) as sess:
    load(saver, sess, epoch=20, path='./model/')
    # 予測正解率を計算
    preds = predict(sess, X_test_centered, return_proba=False)
    print('Test Accuracy: %.3f%%' % (100*np.sum(preds == y_test)/len(y_test)))


Building 1st layer: 
<tf.Variable 'conv_1/_weights:0' shape=(5, 5, 1, 32) dtype=float32_ref>
<tf.Variable 'conv_1/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("conv_1/Conv2D:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/net_pre-activation:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/activation:0", shape=(?, 24, 24, 32), dtype=float32)

Building 2nd layer: 
<tf.Variable 'conv_2/_weights:0' shape=(5, 5, 32, 64) dtype=float32_ref>
<tf.Variable 'conv_2/_biases:0' shape=(64,) dtype=float32_ref>
Tensor("conv_2/Conv2D:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/net_pre-activation:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/activation:0", shape=(?, 8, 8, 64), dtype=float32)

Building 3rd layer:
<tf.Variable 'fc_3/_weights:0' shape=(1024, 1024) dtype=float32_ref>
<tf.Variable 'fc_3/_biases:0' shape=(1024,) dtype=float32_ref>
Tensor("fc_3/MatMul:0", shape=(?, 1024), dtype=float32)
Tensor("fc_3/net_pre-activation:0", shape=(?, 1024), dtype=float32

In [124]:
# 上記の予測結果を10コ可視化する(予測されたラベル & それらの確率)
# precision :: 小数点以下の桁数
# suppress :: 指数表記にする/しない
np.set_printoptions(precision=2, suppress=True)

with tf.Session(graph=g2) as sess:
    load(saver, sess, 
         epoch=20, path='./model/')
        
    print(predict(sess, X_test_centered[:10], 
              return_proba=False))
    
    print(predict(sess, X_test_centered[:10], 
                  return_proba=True))

Loading model from ./model/
INFO:tensorflow:Restoring parameters from ./model/cnn-model.ckpt-20
[7 2 1 0 4 1 4 9 5 9]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [125]:
# 40エポックになるまでモデルを更にトレーニングしてみる
with tf.Session(graph=g2) as sess:
    # モデルをロード
    load(saver, sess, epoch=20, path='./model/')
    
    # 追加の20エポックの訓練
    train(sess,
          training_set=(X_train_centered, y_train), 
          validation_set=(X_valid_centered, y_valid),
          initialize=False,
          epochs=20,
          random_seed=123)
        
    save(saver, sess, epoch=40, path='./model/')
    
    # 予測
    preds = predict(sess, X_test_centered, return_proba=False)
    
    print('Test Accuracy: %.3f%%' % (100*np.sum(preds == y_test)/len(y_test)))

Loading model from ./model/
INFO:tensorflow:Restoring parameters from ./model/cnn-model.ckpt-20
Epoch 01 Training Avg. Loss:   4.240  Validation Acc:   0.992
Epoch 02 Training Avg. Loss:   3.690  Validation Acc:   0.993
Epoch 03 Training Avg. Loss:   4.006  Validation Acc:   0.993
Epoch 04 Training Avg. Loss:   3.141  Validation Acc:   0.992
Epoch 05 Training Avg. Loss:   3.226  Validation Acc:   0.992
Epoch 06 Training Avg. Loss:   3.621  Validation Acc:   0.992
Epoch 07 Training Avg. Loss:   2.602  Validation Acc:   0.993
Epoch 08 Training Avg. Loss:   2.317  Validation Acc:   0.992
Epoch 09 Training Avg. Loss:   2.487  Validation Acc:   0.992
Epoch 10 Training Avg. Loss:   2.124  Validation Acc:   0.992
Epoch 11 Training Avg. Loss:   2.231  Validation Acc:   0.991
Epoch 12 Training Avg. Loss:   1.892  Validation Acc:   0.992
Epoch 13 Training Avg. Loss:   2.035  Validation Acc:   0.991
Epoch 14 Training Avg. Loss:   2.532  Validation Acc:   0.992
Epoch 15 Training Avg. Loss:   1.776